# Cleaning the data

# Makes advanced_welcome_table_summed from Welcome_Table_All_Dates_All_Data.csv

In [18]:
import unicodecsv
import math
import matplotlib.pyplot as plt
from datetime import datetime as dt
import numpy as np
import pandas as pd

def read_csv(filename):
    with open(filename, 'rb') as f:
        reader = unicodecsv.DictReader(f)
        return list(reader)

In [19]:
csv_list = []
welcome_table_june = read_csv('Welcome_Table_All_Dates_All_Data.csv')

In [20]:
#welcome_table_june

In [21]:
def set_to_int(column_name):
    for row in welcome_table_june:
        if row[column_name].strip() == "":
            row[column_name] = 0
        row[column_name] = int(row[column_name])


In [22]:
set_to_int('# Returns')
set_to_int('# New')
set_to_int('# in Group')

In [23]:
def parse_date(date):
    if date == '':
        return None
    else:
        try:
            return dt.strptime(date, '%m/%d/%Y')
        except:
            return dt.strptime(date, '%m/%d/%y')

In [24]:
for row in welcome_table_june:
    try:
        if len(row['Date']) > 4:
            new_date = row['Date']
        row['Date'] = parse_date(new_date)
    except:
        print row['Date']

# Grouping the data by date

In [25]:
from collections import defaultdict

In [26]:
welcome_table_stats_by_date = defaultdict(list)
for row in welcome_table_june:
    welcome_table_stats_by_date[str(row["Date"]).split(' ',1)[0]].append(row)

In [27]:
summed_by_date = {}
for key in welcome_table_stats_by_date:
    total_visitors = 0
    total_new = 0
    total_returns = 0
    weather = ""
    temperature = ""
    local = 0
    fake = False
    for visit in welcome_table_stats_by_date[key]:
        if visit["Zip code"]:
            total_visitors += visit["# in Group"]
            total_new += visit["# New"]
            total_returns += visit["# Returns"]
        else:
            #fake = True
            print "dropping", visit
        if(visit["Zip code"] == "11201"):
            local += visit["# in Group"]
        if (visit["Temperature"]):
            temperature = visit["Temperature"]
    if total_visitors != 0 and not fake:
        summed_by_date[key] = {"total_visitors": total_visitors, "total_new": total_new, "total_returns":total_returns,"from_11201": local, "Temperature": temperature}

dropping {u'# in Group': 17, u'Temperature': u'', u'Zip code': u'', u'Weather': u'', u'# Returns': 17, u'Date': datetime.datetime(2016, 12, 1, 0, 0), u'# New': 0}
dropping {u'# in Group': 4, u'Temperature': u'42', u'Zip code': u'', u'Weather': u'windy', u'# Returns': 0, u'Date': datetime.datetime(2017, 3, 2, 0, 0), u'# New': 4}
dropping {u'# in Group': 4, u'Temperature': u'79', u'Zip code': u'', u'Weather': u'sunny', u'# Returns': 0, u'Date': datetime.datetime(2018, 9, 16, 0, 0), u'# New': 4}
dropping {u'# in Group': 2, u'Temperature': u'85', u'Zip code': u'', u'Weather': u'mostly cloudy', u'# Returns': 0, u'Date': datetime.datetime(2018, 8, 2, 0, 0), u'# New': 2}
dropping {u'# in Group': 2, u'Temperature': u'49', u'Zip code': u'', u'Weather': u'Sunny/windy', u'# Returns': 0, u'Date': datetime.datetime(2016, 11, 12, 0, 0), u'# New': 2}
dropping {u'# in Group': 73, u'Temperature': u'84', u'Zip code': u'', u'Weather': u'sunny', u'# Returns': 44, u'Date': datetime.datetime(2016, 8, 27, 0,

In [28]:
summed_by_date =  pd.DataFrame.from_dict(summed_by_date, orient = 'index' )

# Adding weather data and additional features

In [29]:
#summed_by_date

In [30]:
import json

# load from file:
with open('past_weather.json', 'r') as f:
    try:
        weather_dict1 = json.load(f)
    # if the file is empty the ValueError will be thrown
    except ValueError:
        data = {}
        
with open('past_weather2.json', 'r') as f:
    try:
        weather_dict2 = json.load(f)
    # if the file is empty the ValueError will be thrown
    except ValueError:
        data = {}

In [31]:
summed_by_date.head()

,total_visitors,total_returns,total_new,Temperature,from_11201
2015-08-20,37,0,37,82,0
2015-08-22,80,0,80,86,0
2015-08-23,95,3,92,83,0
2015-08-26,42,0,42,82,0
2015-08-28,16,0,16,82,0


In [33]:
#summed_by_date[summed_by_date.index == '6-Sep']

In [34]:
summed_by_date["avg_visits_per_hour"] = float('nan')
summed_by_date["avg_returns_per_hour"] = float('nan')
summed_by_date["avg_new_per_hour"] = float('nan')
summed_by_date["avg_from_11201_per_hour"] = float('nan')
summed_by_date["ratio_11201"] = float('nan')
summed_by_date["day"] = ""
summed_by_date['week'] = ""
summed_by_date["month"] = 6
summed_by_date["year"] = 2016
summed_by_date["given_temp"] = summed_by_date["Temperature"]
summed_by_date["precipitation"] =  float('nan')
summed_by_date["cloud_cover"] = float('nan')
summed_by_date["snow"] = float('nan')
summed_by_date["rain"] = float('nan')
summed_by_date.total_returns = summed_by_date.total_returns.astype(float)
summed_by_date.total_new = summed_by_date.total_new.astype(float)
summed_by_date.from_11201 = summed_by_date.from_11201.astype(float)
        
for val in summed_by_date.index.values:
    if summed_by_date.loc[val]['total_visitors'] != 0 and summed_by_date.loc[val]['total_returns'] <1 and summed_by_date.loc[val]['total_new'] <1:
        summed_by_date.set_value(val,'total_returns', float('nan'))
        summed_by_date.set_value(val, 'total_new', float('nan'))
    if val != 'None':
        the_date = dt.strptime(val, "%Y-%m-%d")
        if the_date < dt.strptime("2018-04-01", "%Y-%m-%d"):
            weather_dict = weather_dict1
        else:
            weather_dict = weather_dict2
        if the_date < dt.strptime("2016-02-06", "%Y-%m-%d"):
            summed_by_date.set_value(val,'from_11201', float('nan'))
        if the_date.weekday() >= 5:
            hours = 4
        elif the_date.weekday() == 1 or  the_date.weekday() == 3:
            hours = 2
        else:
            hours = 2
        try:
            try:
                if weather_dict[val][0]['precipIntensity'] > 0:
                    if weather_dict[val][0]['precipType'] == 'snow':
                        summed_by_date.set_value(val, 'snow', 1)
                        summed_by_date.set_value(val, 'rain', 0)
                    elif weather_dict[val][0]['precipType'] == 'rain':
                        summed_by_date.set_value(val, 'rain', 1)
                        summed_by_date.set_value(val, 'snow', 0)
                    else:
                        print weather_dict[val][0]['precipType']
                else:
                    summed_by_date.set_value(val, 'snow', 0)
                    summed_by_date.set_value(val, 'rain', 0)
                    
                summed_by_date.set_value(val, 'Temperature', weather_dict[val][0]["temperatureMax"])
                summed_by_date.set_value(val, 'precipitation', weather_dict[val][0]['precipIntensity'])
                summed_by_date.set_value(val, 'cloud_cover', weather_dict[val][0]['cloudCover'])
            except:
                print val, "weather"
            summed_by_date.set_value(val, "day", the_date.weekday())
            summed_by_date.set_value(val, "week", the_date.isocalendar()[1])
            summed_by_date.set_value(val, "month", the_date.month)
            summed_by_date.set_value(val, "year", the_date.year)
            summed_by_date.set_value(val, "ratio_11201", summed_by_date.loc[val]["from_11201"]/(1.0* summed_by_date.loc[val]["total_visitors"]))
            summed_by_date.set_value(val, "avg_visits_per_hour", summed_by_date.loc[val]["total_visitors"]/(1.0*hours))
            summed_by_date.set_value(val, "avg_returns_per_hour", summed_by_date.loc[val]["total_returns"]/(1.0*hours))
            summed_by_date.set_value(val, "avg_new_per_hour", summed_by_date.loc[val]["total_new"]/(1.0*hours))
            summed_by_date.set_value(val, "avg_from_11201_per_hour", summed_by_date.loc[val]["from_11201"]/(1.0*hours))
        except:
            print val

2015-10-03 weather
2108-06-10 weather


# Create csv

In [35]:
summed_by_date.to_csv('advanced_welcome_table_summed.csv', sep =',')